In [1]:
from torch.optim import Adam
import torch.multiprocessing as mp

from engines.AuxTrainer import AuxTrainer
from engines.ClsTrainer import ClsTrainer
from engines.SegTrainer import SegTrainer
from model.loss import getTotalLoss
from data_loader.dataset import getDataloaders
from utils.util import initModel, ddp_setup, ddp_cleanup

from configs import CONF


[INFO] MODEL: UNet_default, DATASET: all, LOSS: segmentation
UNet
util
trainer
dset
main


In [5]:
trainLoader, testLoader = getDataloaders(CONF.GPU_ID)
model = initModel(CONF.GPU_ID)
lossFunc = getTotalLoss
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=CONF.INIT_LR, weight_decay=1e-5)

# if CONF.MODEL == 'AuxNet':
#     AuxTrainer(CONF.GPU_ID, model, lossFunc, optimizer, testLoader, trainLoader).train()
# elif CONF.MODEL == 'DenseNet':
#     ClsTrainer(CONF.GPU_ID, model, lossFunc, optimizer, testLoader, trainLoader).train()
# elif CONF.MODEL == 'UNet':
#     SegTrainer(CONF.GPU_ID, model, lossFunc, optimizer, testLoader, trainLoader).train()
    

    

[INFO] found 1143 examples in the training set...
[INFO] found 201 examples in the test set...
------------------------------------------------------------


In [6]:
SegTrainer(CONF.GPU_ID, model, lossFunc, optimizer, testLoader, trainLoader).train()